# Postal Code for the US

**[Work in progress]**

This notebook adds US zip code, place name, latitude, longitude data to .csv files for ingestion into the Knowledge Graph.

Data source: [GeoNames.org](http://download.geonames.org/export/zip/)

Author: Peter Rose (pwrose@ucsd.edu)

In [1]:
import os
from pathlib import Path
from io import BytesIO
import requests
from zipfile import ZipFile
import pandas as pd

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_IMPORT = Path(os.getenv('NEO4J_IMPORT'))
print(NEO4J_IMPORT)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-9f7418e6-ef5d-4a2d-ae16-29a5a6814849/installation-4.1.0/import


### Download data from GeoNames

In [4]:
zip_url = 'http://download.geonames.org/export/zip/US.zip'

In [5]:
content = requests.get(zip_url)
zf = ZipFile(BytesIO(content.content))

for item in zf.namelist():
    print("File in zip: "+  item)

BadZipFile: File is not a zip file

In [ ]:
columns = ['country code', 'postal code', 'place name', 'admin name1', 'admin code1', 'admin name2', 'admin code2', 
           'admin name3', 'admin code3', 'latitude', 'longitude', 'accuracy']

In [ ]:
df = pd.read_csv(zf.open('US.txt'), names=columns, sep='\t', dtype='str', header=0)
df.fillna('', inplace=True)

In [ ]:
df.head()

In [ ]:
df.query("`postal code` == '92130'")

In [ ]:
df.rename(columns={'postal code': 'zip'}, inplace=True)
df.rename(columns={'place name': 'placeName'}, inplace=True)

Create a GeoNames code for admin2 division

In [ ]:
df['admin2_id'] = df['country code'] + '.' + df['admin code1'] + '.' + df['admin code2']

In [ ]:
df = df[['zip', 'admin2_id', 'placeName', 'latitude', 'longitude']]

In [ ]:
df.query("zip == '92130'")

In [ ]:
df.to_csv(NEO4J_IMPORT / "00o-GeoNamesPostalCode.csv", index=False)

In [ ]:
df.head()